In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!nvidia-smi

Fri Feb  3 18:52:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [4]:
%cd /mydrive/X-Rays/

/content/gdrive/My Drive/X-Rays


In [5]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [6]:
# Defining the constants
train_data_dir = '/content/gdrive/My Drive/X-Rays/dataset/train_images/train_images'
test_data_dir = '/content/gdrive/My Drive/X-Rays/dataset/test_images/test_images/'
labels_path = '/content/gdrive/My Drive/X-Rays/dataset/labels_train.csv'

# Read the train labels
labels = pd.read_csv(labels_path)
labels['class_id'] = labels['class_id'].astype(str)

# Define the data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=labels,
    directory=train_data_dir,
    x_col='file_name',
    y_col='class_id',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory=test_data_dir,
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a custom top layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

Found 4672 validated image filenames belonging to 3 classes.
Found 0 images belonging to 0 classes.
29084464/29084464 [==============================] - 0s 0us/step


In [7]:
# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
# Fit the model
history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)],
    validation_data=test_generator,
    validation_steps=len(test_generator),
    verbose=1
)





<ipython-input-8-cb96c03375b5>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
146/146 [==============================] - ETA: 0s - loss: 0.7915 - accuracy: 0.6685

146/146 [==============================] - 1040s 7s/step - loss: 0.7915 - accuracy: 0.6685
Epoch 2/10
146/146 [==============================] - ETA: 0s - loss: 0.5832 - accuracy: 0.7727

146/146 [==============================] - 90s 613ms/step - loss: 0.5832 - accuracy: 0.7727
Epoch 3/10
146/146 [==============================] - ETA: 0s - loss: 0.5259 - accuracy: 0.7879

146/146 [==============================] - 89s 612ms/step - loss: 0.5259 - accuracy: 0.7879
Epoch 4/10
146/146 [==============================] - ETA: 0s - loss: 0.4988 - accuracy: 0.7960

...

146/146 [==============================] - 89s 610ms/step - loss: 0.4683 - accuracy: 0.8048
Epoch 7/10
146/146 [==============================] - ETA: 0s - loss: 0.4552 - accuracy: 0.8196

146/146 [==============================] - 90s 615ms/step - loss: 0.4552 - accuracy: 0.8196
Epoch 8/10
146/146 [==============================] - ETA: 0s - loss: 0.4465 - accuracy: 0.8159

146/146 [==============================] - 89s 610ms/step - loss: 0.4465 - accuracy: 0.8159
Epoch 9/10
146/146 [==============================] - ETA: 0s - loss: 0.4341 - accuracy: 0.8217

146/146 [==============================] - 90s 613ms/step - loss: 0.4341 - accuracy: 0.8217
Epoch 10/10
146/146 [==============================] - ETA: 0s - loss: 0.4275 - accuracy: 0.8264

146/146 [==============================] - 89s 612ms/step - loss: 0.4275 - accuracy: 0.8264


In [ ]:
# Plot the accuracy and loss curves
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Predict the class of the test images
test_generator.reset()
predictions = model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Save the predictions as a CSV file
results = pd.DataFrame({'file_name': test_generator.filenames, 'predicted_class_id': predicted_classes})
results.to_csv('/content/gdrive/My Drive/X-Rays/dataset/predictions.csv', index=False)


In [ ]:

# Map the class IDs to class names
class_mapping = {
    0: 'No Disease',
    1: 'Bacterial Pneumonia',
    2: 'Viral Pneumonia'
}

results['predicted_class'] = results['predicted_class_id'].map(class_mapping)
results.to_csv('/content/gdrive/My Drive/X-Rays/dataset/predictions_with_classes.csv', index=False)